In [1]:
import numpy as np
import pandas as pd 
import random
import methods
import TimeSeries_Clustering
from tqdm import tqdm 
import matplotlib.pyplot as plt
import pandas as pd
from tslearn.clustering import KShape
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.metrics import sigma_gak, cdist_gak
from sklearn.metrics.cluster import adjusted_rand_score
import random
from tslearn.utils import to_time_series_dataset
from tslearn.clustering import TimeSeriesKMeans
import xgboost
import gc
import Evrecsys
import sys
from itertools import combinations, groupby
from collections import Counter
from scipy import sparse
from lightfm import LightFM
from sklearn.metrics.pairwise import cosine_similarity
import Apriori
import lightfm_form
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import recall_at_k
from lightfm import LightFM
import TimeSeries_Clustering
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
#Import Data
#new_transactions = pd.read_csv('new_merchant_transactions.csv', parse_dates=['purchase_date'])
merchants = pd.read_csv('merchants.csv')
#historical_transactions = pd.read_csv('historical_transactions.csv',parse_dates=['purchase_date'])
data = pd.read_csv('data_cards_0.csv',parse_dates=['purchase_date'])
test = pd.read_csv('test.csv', parse_dates=["first_active_month"])
train = pd.read_csv('train.csv', parse_dates=["first_active_month"])

data=reduce_mem_usage(data)
test = reduce_mem_usage(test)
train = reduce_mem_usage(train)

Mem. usage decreased to 114.20 Mb (45.5% reduction)


## Card Features 

In [3]:
#number of historical transactions
#gdf = historical_transactions.groupby("card_id")
#gdf = gdf["purchase_amount"].size().reset_index()
#gdf.columns = ["card_id", "num_hist_transactions"]
#train = pd.merge(train, gdf, on="card_id", how="left")
#test = pd.merge(test, gdf, on="card_id", how="left")

In [5]:
#number of new transactions
gdf = data.groupby("card_id")
gdf = gdf["purchase_amount"].size().reset_index()
gdf.columns = ["card_id", "num_transactions"]
train = pd.merge(train, gdf, on="card_id", how="left")
test= pd.merge(test, gdf, on="card_id", how="left")

In [6]:
#Stadistics about purchase amount in historic
#gdf = historical_transactions.groupby("card_id")
#gdf = gdf["purchase_amount"].agg(['sum', 'mean', 'std', 'min', 'max']).reset_index()
#gdf.columns = ["card_id", "sum_hist_trans", "mean_hist_trans", "std_hist_trans", "min_hist_trans", "max_hist_trans"]
#train = pd.merge(train, gdf, on="card_id", how="left")
#test = pd.merge(test, gdf, on="card_id", how="left")

In [7]:
#Stadistics about purchase amount in new merch
gdf = data.groupby("card_id")
gdf = gdf["purchase_amount"].agg(['sum', 'mean', 'std', 'min', 'max']).reset_index()
gdf.columns = ["card_id", "sum_trans", "mean_trans", "std_trans", "min_trans", "max_trans"]
train = pd.merge(train, gdf, on="card_id", how="left")
test = pd.merge(test, gdf, on="card_id", how="left")

In [8]:
train["year_first"] = train["first_active_month"].dt.year
test["year_first"] = test["first_active_month"].dt.year
train["month_first"] = train["first_active_month"].dt.month
test["month_first"] = test["first_active_month"].dt.month
data["year_purch"] = new_transactions["purchase_date"].dt.year
new_transactions["month_purch"] = new_transactions["purchase_date"].dt.month

In [9]:
train.shape

(201917, 14)

In [10]:
new_transactions.shape

(1963031, 17)

In [19]:
train_df, val_df = train_test_split(train, test_size=0.3)

In [20]:
train_df=train_df.merge(data, on='card_id', how='left')
train_avgtarget = train_df.groupby(["merchant_id"])["target"].aggregate("mean").reset_index()
train_df=train_df.merge(train_avgtarget, on='merchant_id', how='left')

In [21]:
val_df=val_df.merge(data, on='card_id', how='left')
val_df=val_df.merge(train_avgtarget, on='merchant_id', how='left')

## LIGHTFM

In [14]:
train_df = train_df.fillna(0)
val_df = val_df.fillna(0)
set1 = set(train_df.card_id.unique())
set2 = set(val_df.card_id.unique())

In [15]:
len(train_df.card_id.unique())

125465

In [22]:
grouped_train_i = train_df.groupby(["merchant_id", "card_id"])["target_y"].aggregate("mean").reset_index()

In [23]:
grouped_val_i = test_df.groupby(["merchant_id", "card_id"])["target_y"].aggregate("mean").reset_index()

,merchant_id,card_id,target_y
0,M_ID_0000699140,C_ID_050f94b707,-11.796875
1,M_ID_000087311e,C_ID_252f6288e1,NaN
2,M_ID_0000ab0b2d,C_ID_0217abd104,NaN
3,M_ID_0000ab0b2d,C_ID_a47cc06b40,NaN
4,M_ID_0000edb21f,C_ID_23df3e779d,0.142090
5,M_ID_0000edb21f,C_ID_45f707d480,0.142090
6,M_ID_0000edb21f,C_ID_ff7d6bfcba,0.142090
7,M_ID_0001c38687,C_ID_b784eddabe,0.359863
8,M_ID_0001c38687,C_ID_e99e67f53e,0.359863
9,M_ID_0002299001,C_ID_4cc4b1fdf3,-0.665527


In [24]:
interactions_i = lightfm_form.create_interaction_matrix(df=grouped_train_i,user_col='merchant_id',item_col='card_id',
                                                            rating_col='target_y')

interactions_test_i = lightfm_form.create_interaction_matrix(df=grouped_test_i,
                                                                 user_col='merchant_id',
                                                                 item_col='card_id',
                                                                 rating_col='target_y')

MemoryError: 

In [35]:
missing = pd.DataFrame.from_dict(list(sorted(set1 - set2)))
added = pd.DataFrame.from_dict(list(sorted(set2 - set1)))

for i in range(len(missing)):
    a = missing[0][i]
    test = test.append({'card_id': a}, ignore_index=True)
for i in range(len(added)):
    a = added[0][i]
    train = train.append({'card_id': a}, ignore_index=True)

TypeError: '<' not supported between instances of 'int' and 'str'

In [ ]:
def lightfm(train, test, return_pred, num_cluster):
    set1 = set(train_df.card_id.unique())
    set2 = set(test_df.card_id.unique())
    missing = pd.DataFrame.from_dict(list(sorted(set1 - set2)))
    added = pd.DataFrame.from_dict(list(sorted(set2 - set1)))

    for i in range(len(missing)):
        a = missing[0][i]
        test = test.append({'card_id': a}, ignore_index=True)
    for i in range(len(added)):
        a = added[0][i]
        train = train.append({'card_id': a}, ignore_index=True)

    train = train.fillna(0)
    test = test.fillna(0)

    grouped_train_i = train.groupby(["merchant_id", "card_id"])["target_y"].aggregate("mean").reset_index()
    grouped_test_i = test.groupby(["merchant_id", "card_id"])["target_y"].aggregate("mean").reset_index()

    interactions_i = lightfm_form.create_interaction_matrix(df=grouped_train_i,
                                                            user_col='merchant_id',
                                                            item_col='cart_id',
                                                            rating_col='target_y')

    interactions_test_i = lightfm_form.create_interaction_matrix(df=grouped_test_i,
                                                                 user_col='merchant_id',
                                                                 item_col='cart_id',
                                                                 rating_col='target_y')

    mf_model = lightfm_form.runMF(interactions=interactions_i,
                                  n_components=30, loss='warp', epoch=40, n_jobs=4)

    #test_history = test[test['reordered'] == 1].groupby('order_id').aggregate({'product_id': lambda x: list(x)})
    #test_history = pd.merge(test_history, test[['order_id', 'user_id']], on='order_id')
    #test_history = test_history.drop_duplicates(subset=['order_id', 'user_id'], keep='first')
    #n_users, n_items = interactions_i.shape

    #results = []
    #test_history['pred'] = 0
    #for user_id in test_history['user_id']:
    #    print(user_id)
    #    recom = mf_model.predict(user_id, np.arange(n_items), num_threads=4)
    #    recom = pd.Series(recom)
    #    recom.sort_values(ascending=False, inplace=True)
    #    if (len(results) == 0):
    #        results = np.array(recom.iloc[0:10].index.values)
    #    else:
    #        results = np.vstack((results, recom.iloc[0:10].index.values))

    #results_df = pd.DataFrame(data=results)
    #columns = results_df.columns.values
    #test_history['pred'] = results_df[columns].values.tolist()

    #predict_labels = test_history['pred']
    #y_test = test['target']

    print('FINISH LIGHTFM')
    if return_pred==0:
        return np.sqrt(mean_squared_error(predict_labels, y_test))
    else:
        return np.sqrt(mean_squared_error(predict_labels, y_test)), predict_labels, y_test
    

In [ ]:
def lightfm(train_df, test_df, return_pred=0, num_cluster=0)